# Assignment 6 (Machine Learning)
> This assignment is solved by:-
* Name : Chayan Sinha
* Sub-Group: 
* Roll Number: 102003347

### Question 1

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_table('./sms.tsv', names=['Label', 'Message'])

In [3]:
df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
X = df['Message']
y = df['Label']

In [5]:
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [6]:
def text_process(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    nostop =  [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    ps = PorterStemmer()
    stemmedData = [ps.stem(word) for word in nostop]
    return stemmedData

In [7]:
text_process("Hello! How are you? It's so typical of me to talk about myself.")

['hello', 'typic', 'talk']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [11]:
from sklearn.pipeline import Pipeline

pipelined_model = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB()),
])

In [12]:
pipelined_model.fit(X_train, y_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7facc14a98b0>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', MultinomialNB())])

In [13]:
pred = pipelined_model.predict(X_test)

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

In [15]:
print(confusion_matrix(y_test, pred))

[[1448    0]
 [  60  164]]


In [16]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1448
        spam       1.00      0.73      0.85       224

    accuracy                           0.96      1672
   macro avg       0.98      0.87      0.91      1672
weighted avg       0.97      0.96      0.96      1672



### Question 2

In [65]:
from sklearn.datasets import load_iris

In [66]:
df = load_iris()

In [67]:
X = pd.DataFrame(df.data, columns=df.feature_names)
y = df.target

In [68]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

> i.

In [70]:
# Calculating the class priors
train_size = X_train.shape[0]
class_priors = {}
for outcome in np.unique(y_train):
    outcome_count = sum(y_train == outcome)
    class_priors[outcome] = outcome_count/train_size
print(class_priors)

{0: 0.3333333333333333, 1: 0.3416666666666667, 2: 0.325}


In [71]:
X_train.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')

In [72]:
def gaussianDistribution(x, m, v):
    front = 1/(np.sqrt(2*np.pi*v))
    back = np.exp((-1*np.square(x-m))/(2*v))
    return front*back

In [73]:
X_train_np = X_train.to_numpy()

In [74]:
cluster0 = []
cluster1 = []
cluster2 = []
for i in range(len(X_train)):
    if(y_train[i]==0):
        cluster0.append(X_train_np[i])
    elif(y_train[i]==1):
        cluster1.append(X_train_np[i])
    else:
        cluster2.append(X_train_np[i])
cluster0 = np.array(cluster0)
cluster1 = np.array(cluster1)
cluster2 = np.array(cluster2)

In [75]:
def prob_calc(cluster, x, i):
    m = np.mean([row[i] for row in cluster])
    v = np.var([row[i] for row in cluster])
    return gaussianDistribution(x, m, v)

In [76]:
X_test_np = X_test.to_numpy()

In [77]:
y_pred = []
for i in range(len(y_test)):
    x = X_test_np[i]
    denom = prob_calc(X_train_np, x[0], 0) * prob_calc(X_train_np, x[1], 1) * prob_calc(X_train_np, x[2], 2) * prob_calc(X_train_np, x[3], 3)
    p_0 = class_priors[0] * prob_calc(cluster0, x[0], 0) * prob_calc(cluster0, x[1], 1) * prob_calc(cluster0, x[2], 2) * prob_calc(cluster0, x[3], 3)/denom
    p_1 = class_priors[1] * prob_calc(cluster1, x[0], 0) * prob_calc(cluster1, x[1], 1) * prob_calc(cluster1, x[2], 2) * prob_calc(cluster1, x[3], 3)/denom
    p_2 = class_priors[2] * prob_calc(cluster2, x[0], 0) * prob_calc(cluster2, x[1], 1) * prob_calc(cluster2, x[2], 2) * prob_calc(cluster2, x[3], 3)/denom
    if(p_0>p_1):
        if(p_0>p_2):
            y_pred.append(0)
        else:
            y_pred.append(2)
    else:
        if(p_1>p_2):
            y_pred.append(1)
        else:
            y_pred.append(2)

In [79]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred)*100)

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
100.0


> ii.

In [80]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [81]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
1.0


### Question 3

In [82]:
from sklearn.datasets import load_breast_cancer

In [83]:
X, y = load_breast_cancer(return_X_y=True)

In [89]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
k_range = list(range(1,31))
param_grid = dict(n_neighbors=k_range)

In [87]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [90]:
# cv = cross-validation sets
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False, verbose=1)
grid_search = grid.fit(X_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


In [91]:
print(grid_search.best_params_)

{'n_neighbors': 3}


In [93]:
accuracy = grid_search.best_score_ *100
print(accuracy)

92.69656699889259


In [94]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred) * 100)

93.00699300699301
